In [1]:
import requests 
import pandas as pd 
from bs4 import BeautifulSoup 
import html5lib 
import logging  
import json  
import sqlite3 

# Setup logging
logging.basicConfig(filename='etl_project_log.txt', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

def extract_gdp_data():
    logging.info('Fetching data from Wikipedia')
    url = "https://web.archive.org/web/20230902185326/https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29"
    response = requests.get(url)
    if response.status_code == 200:
        logging.info("Data fetched successfully")
        soup = BeautifulSoup(response.text, "html5lib")
        table = soup.find_all('tbody')[2]
        data = []
        for row in table.find_all('tr'):
            cols = row.find_all('td')
            cols = [ele.text.strip() for ele in cols]
            data.append([ele for ele in cols if ele])  # Get rid of empty values
        df = pd.DataFrame(data)
        logging.info('Data extracted and read into pandas DataFrame')
        return df
    else:
        logging.error(f'Failed to fetch data. Status code: {response.status_code}')
        return None

def transform_gdp_data(df):
    logging.info('Starting data transformation')
    # Assuming the columns you want start from the second column in the DataFrame
    gdp_df = df.iloc[:, [0, 2]].copy()  # Modify indices as per actual data
    gdp_df.columns = ['Country', 'GDP_USD_BILLION']
    gdp_df['GDP_USD_BILLION'] = pd.to_numeric(gdp_df['GDP_USD_BILLION'].str.replace(',', ''), errors='coerce')
    gdp_df = gdp_df.sort_values(by='GDP_USD_BILLION', ascending=False).reset_index(drop=True)
    logging.info('Data transformation completed')
    return gdp_df

def load_data_to_json(df):
    json_file = 'Countries_by_GDP.json'
    df.to_json(json_file, orient='records', indent=2)
    logging.info(f'Data loaded into {json_file}')

def load_data_to_db(df):
    db_file = 'World_Economies.db'
    logging.info(f'Loading data into SQLITE database {db_file}')
    conn = sqlite3.connect(db_file)
    df.to_sql('Countries_by_GDP', conn, if_exists='replace', index=False)
    conn.close()
    logging.info('Data loaded into the Database')

def query_db():
    db_file = 'World_Economies.db'
    query = "SELECT * FROM Countries_by_GDP WHERE GDP_USD_BILLION > 100" 
    logging.info('Querying database for Countries with GDP greater than 100 billion USD')
    conn = sqlite3.connect(db_file)
    result_df = pd.read_sql_query(query, conn)
    conn.close()
    logging.info('Query executed successfully')
    return result_df

def main():
    logging.info('ETL PROCESS STARTED')
    df = extract_gdp_data()
    if df is not None:
        df.columns = ['Country/Territory', 'UN region', 'Estimate 1', 'Year 1', 'Estimate 2', 'Year 2', 'Estimate 3', 'Year 3']  # Set your DataFrame column names
        transformed_df = transform_gdp_data(df)
        load_data_to_json(transformed_df)
        load_data_to_db(transformed_df)
        result_df = query_db()
        print(result_df)
    logging.info('ETL process completed')

if __name__ == '__main__':
    main()


              Country  GDP_USD_BILLION
0               World      105568776.0
1       United States       26854599.0
2               China       19373586.0
3               Japan        4409738.0
4             Germany        4308854.0
..                ...              ...
186        Micronesia            456.0
187  Marshall Islands            291.0
188             Palau            262.0
189          Kiribati            248.0
190             Nauru            151.0

[191 rows x 2 columns]
